In [ ]:
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
import torch
from huggingface_hub import login
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


with open("HF_TOKEN.txt", "r") as f:
    token = f.read().strip()

login(token=token)

In [ ]:
import json
from typing import List, Dict

#chunk archivo json con todo los productos de mercadona.

def generar_chunks_para_vectorizar(ruta_archivo: str) -> List[str]:
    """
    Carga el JSON y crea una lista de strings (chunks) para ser vectorizados,
    combinando categoría, subcategoría y el nombre del producto.
    """
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            datos: List[Dict] = json.load(f)
        
        chunks: List[str] = []
        for categoria_principal in datos:
            nombre_categoria = categoria_principal['name']
            
            for subcategoria in categoria_principal.get('subcategories', []):
                nombre_subcategoria = subcategoria['name']
                
                for producto in subcategoria.get('products', []):
                    chunks.append(f"{nombre_categoria} : {nombre_subcategoria} : {producto}")

        return chunks
    
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return []

chunks_a_vectorizar = generar_chunks_para_vectorizar('./data/Silver/mercadona_data.json')
if not chunks_a_vectorizar:
    print("No se pudieron generar los chunks. Saliendo...")
    exit()

print(f"Se han generado {len(chunks_a_vectorizar)} chunks para vectorizar.")

In [ ]:
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
from torch import Tensor


#carga embedding

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                 
    bnb_4bit_quant_type="nf4",        
    bnb_4bit_compute_dtype=torch.bfloat16, 
    bnb_4bit_use_double_quant=True,   
)



model_name_embedding = 'intfloat/multilingual-e5-large-instruct'  #"Qwen/Qwen3-Embedding-0.6B"  
tokenizer_embedding = AutoTokenizer.from_pretrained(model_name_embedding)
model_embedding = AutoModel.from_pretrained(model_name_embedding,
                                            torch_dtype=torch.bfloat16,
                                            quantization_config=bnb_config,
                                           )


device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model_embedding.to(device)


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


batch_size = 64
dataloader = DataLoader(chunks_a_vectorizar, batch_size=batch_size, shuffle=False)

embeddings_array = []


with torch.no_grad():
    for batch_texts in tqdm(dataloader, desc="Vectorizando batches"):
        batch_dict = tokenizer_embedding(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        ).to(device)

        outputs = model_embedding(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        embeddings_normalized = F.normalize(embeddings, p=2, dim=1)
        
        embeddings_array.append(embeddings_normalized.cpu())

embeddings_tensor = torch.cat(embeddings_array, dim=0)

embeddings_tensor.shape

In [ ]:
from typing import List

# La instrucción   el modelo según la documentación
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery:{query}'

def buscar_entradas_relevantes(
    producto: str,
    embeddings_db: torch.Tensor,
    chunks: List[str],
    top_k: int = 5
) -> List[str]:
    """
    Busca las entradas más similares al producto en la base de datos de embeddings.
    
    Args:
        producto (str): El nombre del producto a buscar.
        embeddings_db (torch.Tensor): El tensor con los embeddings de la base de datos.
        chunks (List[str]): La lista original de textos que se vectorizaron.
        top_k (int): El número de resultados más relevantes a devolver.
    
    Returns:
        List[str]: Las k entradas más relevantes.
    """
    # 1. Prepara la consulta con la instrucción
    task = 'Given a product, retrieve relevant categories from a grocery store'
    consulta_con_instruct = get_detailed_instruct(task, producto)
    
    # 2. Vectoriza la consulta
    with torch.no_grad():
        batch_dict = tokenizer_embedding(
            [consulta_con_instruct],
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        ).to(device)
        
        outputs = model_embedding(**batch_dict)
        embedding_producto = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        embedding_producto_normalized = F.normalize(embedding_producto, p=2, dim=1).cpu()

    # 3. Calcula la similitud del coseno entre la consulta y la base de datos
    puntuaciones_similitud = torch.matmul(embedding_producto_normalized, embeddings_db.T)[0]
    
    # 4. Obtiene los índices de las top_k entradas más similares
    indices_top_k = torch.topk(puntuaciones_similitud, k=top_k).indices.tolist()

    # 5. Devuelve los chunks originales correspondientes a esos índices
    entradas_encontradas = [chunks[i] for i in indices_top_k]
    return entradas_encontradas


def buscar_en_prompt(lista):
    resultados = []
    try:
        for i, item in enumerate(lista.split("1:")[1].split(":")):  
            resultados.append(f"estos son los resultados para : {item}")
            a = buscar_entradas_relevantes(producto=item, embeddings_db=embeddings_tensor, chunks=chunks_a_vectorizar, top_k = 10)
            resultados.append(a)
            resultados.append("Indeterminado : Indeterminado : Productos no listados")
    except:
        resultados = "Indeterminado : Indeterminado : Productos no listados"
    return resultados


    

In [ ]:
#Load data
df = pd.read_parquet('./data/Silver/Cleaned_data.parquet')


### para hacer prototipado de las querys. 
#df = df.sample(n=20)


###subset aleatorio pero manteniendo proporcion de busqueda mercadona-hacendado
n = 10000
df_balanceado = (df.groupby('search', group_keys=False).apply(lambda x: x.sample(n=n//2, random_state=69)))

df = df_balanceado 

# indices_a_conservar = [ 301265, 152950 ]
# df = df.loc[df.index.isin(indices_a_conservar)]
# df


model_name = "google/t5gemma-9b-9b-ul2-it"

## Load the model T5-gemma
# Configuración de la cuantificación de 4 bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                 
    bnb_4bit_quant_type="nf4",        
    bnb_4bit_compute_dtype=torch.bfloat16, 
    bnb_4bit_use_double_quant=True,   
)


## al tener dos GPU de 12 G tengo que dividir manualmente el modelo porque el "auto" lo fragmenta
# y a la hora de generar el output genera error
device_map = {
    'model.encoder': 'cuda:0',
    'model.decoder': 'cuda:1',
    'lm_head': 'cuda:1',
    'model.shared': 'cuda:0' 
}


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map=device_map, ## Normalmente "auto" o cuda:0, cuda:1 
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
)

In [ ]:
### querys secuenciales, el salto de query es para usar ese indice 

querys= [
    "Clasifica el sentimiento del siguiente texto como 'positivo', 'negativo' o 'neutral'. Considera que algunos mensajes están escritos con humor. Explica brevemente tu decisión en este formato:  Sentimiento: [tu clasificación]. Razón: [tu explicación en menos de 50 palabras].",
    #f"Justifica brevemente, 10-20 palabras, el porqué se ha clasificado con este sentimiento o connotación  '{sentimiento}' este texto: " ,
    
    
    #"Clasifica el sentimiento del siguiente texto en 'positivo', 'negativo' o 'neutral' y explica el porqué en una frase corta. Sigue este formato: 'Sentimiento: [tu clasificacion]. Razón: [tu explicacion]'. No excedas las 20 palabras en la razón. Ten en cuenta que son mensajes tipo tweet. Piensa paso a paso",  
    "Clasifica el sentimiento del siguiente texto como 'positivo', 'negativo' o 'neutral'. Ten en cuenta que muchos mensajes son informales, sin puntuación, o escritos en mayúsculas. No asumas un tono emocional solo por eso. Detecta y considera humor, sarcasmo, ironía, exageraciones o contradicciones (por ejemplo: 'en verdad no', 'ya claro', 'sí pero no', uso excesivo de signos o mayúsculas). Determina el sentimiento real que transmite el mensaje. Considera como neutral todo mensaje que simplemente informe un hecho o dato sin mostrar emociones claras.  Explica brevemente tu decisión en este formato:  Sentimiento: [tu clasificación]. Razón: [tu explicación en menos de 50 palabras]. Piensa paso a paso antes de decidir.",
    
    
    "Analiza si el texto contiene información personal (teléfono, dirección, DNI). Responde solo con 'SI' o 'NO'.",

    
    #"Analiza si el texto menciona un producto que pueda venderse en un supermercado, incluyendo alimentos, bebidas, productos de limpieza, artículos para el hogar, higiene o cualquier otro artículo. 'Mercadona' o 'Hacendado' no son productos, sino el nombre del supermercado o la marca. Si menciona un producto, responde solo con 'SI'; si no, responde 'NO'.",
    "Analiza si el texto menciona un producto que pueda venderse en un supermercado, incluyendo alimentos, bebidas, productos de limpieza, artículos para el hogar, higiene personal, cosmética, maquillaje, skincare u otras categorías similares. Si menciona un producto o una categoría de productos, responde solo con 'SI'; si no, responde 'NO'.",

    "Salto de query", # para usar prompt custom 
    
    #"Analiza si el texto menciona uno o más productos o categorías que puedan venderse en un supermercado, incluyendo alimentos, bebidas, limpieza, hogar, higiene personal, cosmética, maquillaje, skincare u otros similares. 'Mercadona' y 'Hacendado' no son productos. Si menciona un producto o categoría, devuelve el nombre exacto tal como aparece en el texto, eliminando cualquier referencia a 'Mercadona' o 'Hacendado'. Si hay varios, usa el formato '1: Producto_1, 2: Producto_2'. Si no menciona ninguno, responde exactamente con 'No aplica'.",
    
    #"Analiza si el texto menciona uno o más productos o categorías de productos que puedan venderse en un supermercado. Esto incluye no solo artículos específicos como 'leche' o 'detergente', sino también categorías generales como 'skincare', 'cuidado de la piel', 'maquillaje', 'cosmética', 'higiene personal', 'productos de limpieza' o equivalentes. Considera sinónimos y términos en otros idiomas que signifiquen lo mismo. 'Mercadona' o 'Hacendado' no son productos. Si menciona uno o varios, responde con el nombre exacto tal como aparece en el texto (sin añadir 'de Mercadona', 'del Mercadona', 'de Hacendado' o 'del Hacendado'). Si hay varios, usa el formato '1: Producto_1, 2: Producto_2'. Si no menciona ninguno, responde exactamente con 'No aplica'.",
    #"Extrae del texto productos o categorías de supermercado. Esto incluye no solo artículos específicos como 'leche' o 'detergente', sino también categorías generales como 'skincare', 'cuidado de la piel', 'maquillaje', 'cosmética', 'higiene personal', 'productos de limpieza' o equivalentes. Considera sinónimos y términos en otros idiomas que signifiquen lo mismo. 'Mercadona', 'Hacendado', 'mercadona' o 'hacendado' no son productos. responde con el nombre exacto tal como aparece en el texto (sin añadir 'de Mercadona', 'del Mercadona', 'de Hacendado' o 'del Hacendado'). Responde 'No aplica' si no hay productos. Si hay uno o varios, lista los nombres exactos tal como aparecen, en formato 1: Producto_1, 2: Producto_2.",
    """Extrae del texto productos o categorías de supermercado.

        Reglas y pasos a seguir:
        
        1. **Regla prioritaria antes de extraer**: 
           - Elimina de la consideración toda mención a nombres de tiendas, cadenas o marcas si no vienen acompañadas de un producto específico.
           - Ejemplos: “Mercadona”, “Hacendado” no se consideran productos.
           - Solo si aparece un producto junto a la marca se incluye (por ejemplo: “galletas Hacendado”).
        
        2. Lee el texto de entrada con atención.
        
        3. Identifica el producto de mercadona o hacendado:
           - Incluye artículos específicos como “leche”, “detergente”, “pan integral”.
           - Incluye categorías generales como “skincare”, “cuidado de la piel”, “maquillaje”, “cosmética”, “higiene personal”, “productos de limpieza”, o equivalentes.
           - Considera sinónimos y términos en otros idiomas que signifiquen lo mismo.
           - Considera sabores, colores y adjetivos inseparables como parte del producto, por ejemplo: “jamón serrano”, “patatas sabor jamón”, “donuts de chocolate”.
           - No incluyas adjetivos sueltos que no sean parte inseparable del producto.
        
        4. No fragmentar productos compuestos:
           - Si un producto está formado por varios elementos que aparecen juntos y funcionan como un solo producto, listarlo como una única entrada. Ejemplo: “conos de bacon y queso” → solo un producto.
        
        5. Excluir elementos que no sean productos o categorías reales:
           - No incluir marcas, tiendas o cadenas sin producto específico.
           - No incluir frases como “de Mercadona” o “de Hacendado”.
           - No incluir menciones de productos usadas solo en sentido figurado.
           - No incluir adjetivos, descriptores o calificativos que no formen parte inseparable del nombre del producto.
        
        6. Mantener el nombre exacto tal como aparece en el texto:
           - No corregir, traducir o modificar.
           - Respetar mayúsculas, minúsculas y ortografía original.
        
        7. Eliminar duplicados y genéricos si existen versiones más específicas:
           - Si un producto está completamente contenido en otro más largo, mantener solo el más específico.
           - Ejemplo: si aparecen “tinte” y “tinte pelirrojo”, incluir solo “tinte pelirrojo”.
        
        8. Formato de respuesta:
           - Si no hay productos. Responde exactamente: No aplica
           - Si hay uno o más productos. listarlos en formato:
             1: Producto_1
             2: Producto_2
             3: Producto_3
             (manteniendo el orden en el que aparecen en el texto).
        
        9. Paso final de control:
           - Confirmar que:
             - No hay marcas o tiendas solas.
             - No hay duplicados.
             - No hay versiones genéricas si hay una más específica.
             
         Ejemplo: 
         Texto: 'MADRE MIA, QUE BUENA ESTA LA NOCILLA DEL MERCADONA' 
         Respuesta:
         1: Nocilla
         
         -Ahora analiza:""",
    
    "Salto de query", # para usar prompt custom 
    "Salto de query", # para usar prompt custom 
    "Salto de query", # para usar prompt custom 
    "Salto de query", # para usar prompt custom 
    "Salto de query",


]

In [ ]:
## prompts y generacion 

df_resultado = pd.DataFrame()

for documento in tqdm(df['tweets'], desc="Processing Tweets"):
    # print("===============================================")
    # print("===============================================")
    # print("===============================================")
    #print(documento)

    # #dataframe temporal y listas temporales
    temp_df = pd.DataFrame()
    explica_sentimiento, explica_sentimiento_2, LOPD, integridad_mensaje, deteccion_producto, sentimiento_final, producto, sentimiento_producto, imagen_marca, categoria, comparativa_producto, comparativa_sentimiento_producto = [], [], [], [], [], [], [], [], [], [], [], []

    
    for i, user_query in enumerate(querys):     
        # Crear prompt

        ##para acelerar el procesado, se verifica si hay alggun producto detectado, igual en el i==6 
        if i == 5 and deteccion_producto[-1].strip()  == 'NO':
            producto.append('No aplica')
            
        else:
            
            if i == 0 or i == 1 or i == 2 or i == 3 or i==5:     
                prompt = [
                    {
                        "role": "user",
                        "content": (
                            "Eres un analista experto en mensajes breves. Tu única tarea es extraer información explícita del mensaje.\n"
                            "Responde **solo** con el valor solicitado. No añadas introducciones, explicaciones, encabezados, ni texto adicional.\n"
                            "No uses formato, como negritas, cursivas o cualquier otro tipo de markdown.\n"
                            "Utiliza el formato específico de la pregunta (ej. 'SI'/'NO', 'positivo'/'negativo'/'neutral').\n"
                            "Responde siempre en español.\n"
                            "Esfuerzate al 100%.\n"
                            "Los resultados son muy importantes para mi.\n"
                            f"Tarea: {user_query}\n"
                            f"Texto: {documento}"
                        )
                    }
        
                ]
        
        
                #Generar respuesta
                input_ids = tokenizer.apply_chat_template(prompt, return_tensors="pt", return_dict=True, add_generation_prompt=True)    
                input_ids = {k: v.to('cuda:0') for k, v in input_ids.items()}
                outputs = model.generate(**input_ids, max_new_tokens=100, do_sample=False) #determinista temperatura 0 y do_sample
                response = tokenizer.decode(outputs[0],skip_special_tokens=True)
                #print(response)

            else:
                #print('saltamos query list')
                pass
  
            # Almacenar en la variable correspondiente
            if i == 0:
                explica_sentimiento.append(response)
            elif i == 1:
                explica_sentimiento_2.append(response)
            elif i == 2:
                LOPD.append(response)
            elif i == 3:
                deteccion_producto.append(response)
            elif i == 5:
                producto.append(response)
            
            elif i == 4:
                prompt_desempate = [{
                    "role": "user",
                    "content": (
                        "Eres un analista experto en mensajes breves. Tu única tarea es extraer información explícita del mensaje.\n"
                        "Responde **solo** con el valor solicitado. No añadas introducciones, explicaciones, encabezados, ni texto adicional.\n"
                        "No uses formato, como negritas, cursivas o cualquier otro tipo de markdown.\n"
                        "Utiliza el formato específico de la pregunta (ej. 'positivo'/'negativo'/'neutral').\n"
                        "Responde siempre en español.\n"
                        "Esfuerzate al 100%.\n"
                        "Los resultados son muy importantes para mi.\n"
                        f"Tarea: Debes desempatar la clasificación de sentimiento para el texto. El primer veredicto es '{explica_sentimiento}' y el segundo es '{explica_sentimiento_2}'. Analiza cuidadosamente el texto completo, y las justificaciones, la ironía, el humor o el sarcasmo son importantes para tomar una decisión. Piensa paso a paso de forma interna pero no muestres tu razonamiento. Responde únicamente con una sola palabra: 'positivo', 'negativo' o 'neutral'. No añadas explicaciones, ejemplos ni justificaciones. Texto a analizar: '{documento}'."
                    )}]
    
    
                input_ids = tokenizer.apply_chat_template(prompt_desempate, return_tensors="pt", return_dict=True, add_generation_prompt=True)    
                input_ids = {k: v.to('cuda:0') for k, v in input_ids.items()}
                outputs = model.generate(**input_ids, max_new_tokens=10, do_sample=False) 
                response_desempate = tokenizer.decode(outputs[0],skip_special_tokens=True)
                
                #print(response_desempate)
    
                sentimiento_final.append(response_desempate)
                 
            elif i == 6:
                
                if deteccion_producto[-1].strip()  == 'NO':
                    sentimiento_producto.append('NO')
    
                else:
                    prompt_producto = [{
                        "role": "user",
                        "content": (
                            "Eres un analista experto en mensajes breves. Tu única tarea es extraer información explícita del mensaje.\n"
                            "Responde **solo** con el valor solicitado. No añadas introducciones, explicaciones, encabezados, ni texto adicional.\n"
                            "No uses formato, como negritas, cursivas o cualquier otro tipo de markdown.\n"
                            "Utiliza el formato específico de la pregunta (ej. 'SI'/'NO').\n"
                            "Responde siempre en español.\n"
                            "Esfuerzate al 100%.\n"
                            "Los resultados son muy importantes para mi.\n"
                            
                            f"""Determina si las justificaciones del sentimiento se refieren al producto '{producto}'.
                            
                            -Instrucciones:
                            1. Lee el texto de entrada y las justificaciones de sentimiento dadas.
                            2. Analiza cada justificación:
                               - Determina si describen una emoción causada directamente por el producto.
                               - Se consideran válidas emociones como gusto, asco, placer, amor, odio, decepción, disgusto, satisfacción, etc., cuando son provocadas por el producto en sí.
                               - Ignora emociones originadas por resistirse al producto, evitarlo o mencionarlo solo en contexto (como parte de una rutina o anécdota).
                               - La necesidad puede estar motivada por una situación negativa, pero eso **no indica una emoción causada por el producto** si no ha sido utilizado o evaluado aún.
                               -Las justificaciones dadas explican únicamente el sentimiento percibido en el texto, no si este está relacionado directamente con el producto.
                               - Tu tarea es decidir si ese sentimiento ha sido causado directamente por el producto.
                            3. Recuerda: 'mercadona', 'Mercadona', 'hacendado' o 'Hacendado' **no son productos por sí solos**.
                               - Pero si se menciona un producto real acompañado de estos términos (ej. “galletas de Hacendado”), **sí se considera válido**.
                            4. Regla de decisión:
                               - Si ambas justificaciones se refieren claramente al producto y la emoción es causada directamente por él, responde: SI.
                               - En cualquier otro caso, responde: NO.
                            5. Formato:
                               - Solo responde con SI o NO, sin comillas ni explicaciones.

                            -Ejemplos:
                            
                            Ejemplo 1:
                            Producto: '1: arroz 2: alga nori'  
                            Texto: 'Quiero hacer sushi pero @Mercadona tuvo la genial idea de quitar el arroz y el alga nori.'
                            Justificación 1: 'Sentimiento: negativo. Razón: El usuario expresa frustración por la falta de ingredientes para hacer sushi, lo que denota un sentimiento negativo.'
                            Justificación 2: 'Sentimiento: negativo. Razón: El mensaje expresa frustración por la falta de ingredientes para hacer sushi, atribuyéndola a una "genial idea" de Mercadona, lo que implica sarcasmo y enfado.'
                            respuesta: SI

                            Ejemplo 2:
                            Producto: '1: lentejas precocinadas'
                            Texto: 'Las lentejas precocinadas me dieron un asco horrible. No pude ni terminarlas.'
                            Justificación 1: 'Sentimiento: negativo. Razón: El autor manifiesta asco y rechazo por la experiencia al consumir las lentejas.'
                            Justificación 2: 'Sentimiento: negativo. Razón: El mensaje refleja disgusto directo tras probar el producto.'
                            Respuesta: SI
                            
                            Ejemplo 3:
                            Producto: '1: bollería'
                            Texto: Ese aire de superioridad con el que paseo por los pasillos de bollería del Mercadona, sin coger nada.'
                            Justificación 1: 'Sentimiento: positivo. Razón: El autor se siente satisfecho por su autocontrol y capacidad de resistirse a la tentación.'
                            Justificación 2: 'Sentimiento: positivo. Razón: El mensaje transmite orgullo personal vinculado al comportamiento del hablante.'
                            Respuesta: NO
                            
                            Ejemplo 4:
                            Producto: '1: Coca Cola Zero'
                            Texto: 'No seré yo quien se meta con Mercadona por poner los precios que le de la gana pero que no engañen. Hasta hace cosa de semana y media compraba un pack de Cocal Cola Zero por 3,7... el pico exacto no me acuerdo. De repente lo subieron a 4€ hace unos días.'                          
                            Justificación 1: 'Sentimiento: negativo. Razón: El usuario expresa su descontento por el aumento de precio de un producto en Mercadona, utilizando un tono crítico.' 
                            Justificación 2: 'Sentimiento: negativo. Razón: El usuario expresa su descontento por el aumento de precio de un producto en Mercadona, utilizando términos como "no engañen" y "de repente".'
                            Respuesta: SI
                            
                            Ejemplo 5: 
                            Producto: '1: repelente de mosquitos' 
                            Texto: 'Tercera noche consecutiva de caza de mosquitos. Esto mañana se acaba porque pienso comprar todo lo habido y por haber que repela a los mosquitos de Mercadona. Así que @Mercadona prepara mercancía que voy.'
                            Justificación 1: 'Sentimiento: negativo. Razón: El usuario se queja de la caza de mosquitos durante tres noches consecutivas y expresa su intención de comprar repelentes.'
                            Justificación 2: 'Sentimiento: negativo. Razón: El mensaje expresa frustración por las mosquitos y la necesidad de comprar repelentes, lo que indica una experiencia negativa.'
                            Respuesta: NO
                            
                            -Ahora analiza el siguiente caso:
                            
                            Producto: '{producto}'
                            Texto: '{documento}'
                            Justificación 1: '{explica_sentimiento}'
                            Justificación 2: '{explica_sentimiento_2}'
                            Respuesta:"""
        
                        )}]
        
                    input_ids = tokenizer.apply_chat_template(prompt_producto, return_tensors="pt", return_dict=True, add_generation_prompt=True)    
                    input_ids = {k: v.to('cuda:0') for k, v in input_ids.items()}
                    outputs = model.generate(**input_ids, max_new_tokens=7, do_sample=False) 
                    response_producto = tokenizer.decode(outputs[0],skip_special_tokens=True)
        
                    #print(response_producto)
        
                    sentimiento_producto.append(response_producto)
    
            elif i == 7:
                prompt_marca = [{
                    "role": "user",
                    "content": (
                        "Eres un analista de mensajes breves experto en marketing. Tu única tarea es extraer información explícita del mensaje.\n"
                        "Responde **solo** con el valor solicitado. No añadas introducciones, explicaciones, encabezados, ni texto adicional.\n"
                        "No uses formato, como negritas, cursivas o cualquier otro tipo de markdown.\n"
                        "Utiliza el formato específico de la pregunta (ej. 'SI'/'NO').\n"
                        "Responde siempre en español.\n"
                        "Esfuerzate al 100%.\n"
                        "Los resultados son muy importantes para mi.\n"

                        f"""Determina si las justificaciones de sentimiento se refieren directamente a la imagen, reputación o percepción de la marca 'Mercadona' o 'Hacendado' en el texto.
                        
                        Instrucciones paso a paso:
                        1. Lee el texto de entrada y las justificaciones de sentimiento dadas.
                        
                        2. Analiza el primer sentimiento y su justificación.
                           - Determina si se refiere directamente a la imagen, reputación o percepción de la cadena de supermercado 'Mercadona' o la marca de alimentos 'Hacendado'.
                           - Considera menciones literales, sinónimos, referencias implícitas o alusiones indirectas a la imagen de marca.
                       
                        3. Analiza el segundo sentimiento y su justificación con el mismo criterio.
                       
                        4. Regla de decisión:
                           - Si una o ambas justificaciones se refieren claramente a la imagen de mercadona o percepción de la marca hacendado, responde exactamente: SI.
                           - Si no lo hacen, responde exactamente: NO.
                       
                        5. Formato de respuesta:
                           - Responde únicamente con SI o NO, sin comillas ni explicaciones.
                           - El razonamiento debe hacerse internamente, pero no mostrarse en la salida final.   

                        -Ejemplos:
                        
                        Ejemplo 1:
                        Texto:'callate zorra te bebes vodka de 3 euros del Mercadona, la vacuna no te va a hacer nada'
                        Primer Sentimiento y su justificación: 'Sentimiento: negativo. Razón: El mensaje es agresivo y despectivo, utilizando insultos y amenazas. No hay indicio de humor o positivismo.'                        
                        Segundo sentimiento y su justificación: 'Sentimiento: negativo. Razón: El mensaje es agresivo y despectivo, utilizando insultos y un tono sarcástico.'
                        Respuesta: 'NO'

                        Ejemplo 2:
                        Texto: 'Al Mercadona le han pillado con el carrito del helado. Los plátanos de canarias los llegué a ver a 3.15€ a unos meses, hoy 2.79€. No venderían uno teniendo las bananas a casi un tercio de precio.'
                        Primer Sentimiento y su justificación: 'Sentimiento: negativo. Razón: El mensaje expresa frustración por el precio de los plátanos de Canarias, a pesar de que han bajado, ya que aún son más caros que las bananas.' 
                        Segundo Sentimiento y su justificación: 'Sentimiento: negativo. Razón: El mensaje expresa sorpresa y desconfianza hacia la bajada de precio de los plátanos, sugiriendo que es una estrategia para vender más plátanos de Canarias.' 
                        Respuesta: 'SI'

                        Ejemplo 3:
                        Texto:'odio que cambien las cosas de sitio en el mercadona cada 2x3, doy vueltas como una tonta'
                        Primer Sentimiento y su justificación: 'Sentimiento: negativo. Razón: El mensaje expresa frustración y molestia por el cambio constante de productos en el supermercado, lo que genera una experiencia negativa para la persona.' 
                        Segundo Sentimiento y su justificación:  'Sentimiento: negativo. Razón: La frase expresa una fuerte desaprobación por el cambio constante de productos en el supermercado, lo que genera frustración y molestia en la persona.' 
                        Respuesta: 'SI'

                        Ejemplo 4:
                        Texto: 'En fin... El Mercadona'
                        Primer Sentimiento y su justificación: 'Sentimiento: neutral. Razón: El mensaje expresa una conclusión, pero no una opinión positiva o negativa sobre Mercadona.'                    
                        Segundo Sentimiento y su justificación: 'Sentimiento: negativo. Razón: El mensaje expresa una frustración o cansancio, como si el usuario estuviera harto de Mercadona.'
                        Respuesta: 'SI'
                        
                        -Ahora analiza el siguiente caso:
                        Texto: '{documento}'
                        Primer Sentimiento y su justificación: '{explica_sentimiento}'
                        Segundo Sentimiento y su justificación: '{explica_sentimiento_2}'
                        Respuesta: """
     
                    )}]
    
    
                input_ids = tokenizer.apply_chat_template(prompt_marca, return_tensors="pt", return_dict=True, add_generation_prompt=True)    
                input_ids = {k: v.to('cuda:0') for k, v in input_ids.items()}
                outputs = model.generate(**input_ids, max_new_tokens=7, do_sample=False) 
                response_marca= tokenizer.decode(outputs[0],skip_special_tokens=True)
    
                #print(response_marca)
    
                imagen_marca.append(response_marca)          


            elif i == 8:
                if deteccion_producto[-1].strip()  == 'NO' or sentimiento_producto[-1].strip()  == 'NO':
                    categoria.append('No aplica') 
                else:               
                    prompt_categoria = [{
                        "role": "user",
                        "content": (
                        f"""Eres un clasificador de productos.
                            Responde **solo** con la categoría y subcategoría exactas de la lista dada.
                            No añadas introducciones, explicaciones, encabezados, ni texto adicional.
                            No uses formato como negritas, cursivas o cualquier tipo de markdown.
                            
                            Tu tarea es clasificar el/los siguientes producto(s) indicados en 'Objeto',
                            basándote en la lista de referencia y usando también la información de 'CONTEXTO'.
                            
                            ---
                            **Instrucciones:**
                            
                            1.  Lee el 'Objeto' para identificar el producto y el 'CONTEXTO' para obtener pistas adicionales.
                            2.  **PRIORIDAD DEL CONTEXTO:** El **CONTEXTO** tiene prioridad absoluta sobre el nombre del producto o la lista de referencia. Si el contexto contiene información clave (ej. "en farmacia", "supermercado", "marca X"), úsala como la pista principal para la clasificación, incluso si otros productos de la lista parecen similares por el nombre.
                            3.  Usa la información del contexto para corregir errores o ambigüedades en el nombre del producto.
                            4.  **Encuentra la coincidencia más cercana:** De toda la lista de referencia, busca la línea que contenga el nombre del producto o su variante más parecida. Esta será tu clasificación principal.
                            5.  Busca la categoría y subcategoría que mejor se ajusten al producto identificado en la lista de referencia.
                                -La lista de referencia sigue el siguiente formato CATEGORIAS : SUBCATEGORIAS : PRODUCTOS
                                
                            6.  Si el producto podría encajar en varias categorías, elige la más probable **según el CONTEXTO**.
                            7.  Si el producto no aparece o no encaja en ninguna categoría, responde con: Indeterminado : Indeterminado.
                            8.  Si se están comparando dos productos entre si, asume que son de la misma categoría y asignalos al más claro. 
                            9.  Devuelve la respuesta en este formato, manteniendo el orden original de aparición:
                                1: Categoria_1 : Subcategoria
                                2: Categoria_2 : Subcategoria
                                ...
                            10.  No incluyas comillas, texto adicional ni inventes categorías.
                            11. Devuelve las categorías y subcategorías de manera completa. Si encuentras la categoría pero hay varias subcategorías posibles, elige la más probable para el producto en función del **CONTEXTO**.

                            Ejemplo 1:
                            Objeto:
                            \n                            \'[\'1: Bephantol\\n2: vaselina \\n\']\'\n                            
                            CONTEXTO:
                            'Bephantol 8,95 en farmacia, vaselina 2 en mercadona, yo no me la juego'
                            Lista de referencia (Categoría : Subcategoría : Producto):
                            '[\'estos son los resultados para :  Bephantol\\n2\', [\'Azúcar, caramelos y chocolate : Azúcar y edulcorante : Edulcorante Eritritol y Sucralosa Hacendado\', \'Cuidado facial y corporal : Higiene bucal : Dentífrico Bicarbonato blanqueador Deliplus\', \'Azúcar, caramelos y chocolate : Azúcar y edulcorante : Edulcorante en pastillas sacarina Hacendado\', \'Azúcar, caramelos y chocolate : Chicles y caramelos : Caramelos eucaliptus mentol Respiral Halls\', \'Cuidado facial y corporal : Gel y jabón de manos : Gel de baño avena Deliplus piel sensible\', \'Cuidado facial y corporal : Gel y jabón de manos : Gel aceite de baño Deliplus piel atópica\', \'Cuidado facial y corporal : Gel y jabón de manos : Jabón de manos avena Deliplus líquido\', \'Azúcar, caramelos y chocolate : Azúcar y edulcorante : Edulcorante líquido sacarina Hacendado\', \'Cuidado facial y corporal : Higiene bucal : Dentífrico Blanqueador Bicarbonato Signal\', \'Azúcar, caramelos y chocolate : Azúcar y edulcorante : Edulcorante granulado stevia Hacendado\'], \'Indeterminado : Indeterminado : Productos no listados\', \'estos son los resultados para :  vaselina \\n\', [\'Maquillaje : Labios : Vaselina hidratante Deliplus\', \'Cuidado facial y corporal : Gel y jabón de manos : Gel de baño vainilla y miel Deliplus piel normal\', \'Maquillaje : Labios : Vaselina perfumada para labios Deliplus frambuesa\', \'Cuidado facial y corporal : Gel y jabón de manos : Esponja de baño flor Deliplus exfoliación suave\', \'Cuidado facial y corporal : Higiene íntima : Toallitas íntimas Deliplus monodosis\', \'Cuidado facial y corporal : Perfume y colonia : Agua de colonia Deliplus Floral Infusion\', \'Fitoterapia y parafarmacia : Parafarmacia : Vaselina hidratante Deliplus\', \'Cuidado facial y corporal : Gel y jabón de manos : Esponja de baño suave Deliplus\', \'Cuidado facial y corporal : Gel y jabón de manos : Jabón de manos avena Deliplus líquido\', \'Cuidado facial y corporal : Gel y jabón de manos : Esponja de baño rizo suave Deliplus\'], \'Indeterminado : Indeterminado : Productos no listados\']'                  
                           
                            Respuesta:
                            1: Fitoterapia y parafarmacia : Parafarmacia
                            2: Fitoterapia y parafarmacia : Parafarmacia
                            

                            Ahora analiza el siguiente caso:
                            Objeto:
                            '{producto}'
                            CONTEXTO:
                            '{documento}'
                            Lista de referencia (Categoría : Subcategoría : Producto):
                            {buscar_en_prompt(producto[-1])}
                    
                            Respuesta:"""
                        )}]
        
                    input_ids = tokenizer.apply_chat_template(prompt_categoria, return_tensors="pt", return_dict=True, add_generation_prompt=True)    
                    input_ids = {k: v.to('cuda:0') for k, v in input_ids.items()}
                    outputs = model.generate(**input_ids, max_new_tokens=50, do_sample=False) 
                    response_categoria = tokenizer.decode(outputs[0],skip_special_tokens=True)
        
                    #print(response_categoria)
        
                    categoria.append(response_categoria)       

            elif i == 9:
                if deteccion_producto[-1].strip()  == 'NO' or sentimiento_producto[-1].strip()  == 'NO':
                    comparativa_producto.append('No aplica') 
                else:
                    prompt_comparativa_producto = [{
                        "role": "user",
                        "content": (
                            "Eres un analista experto en mensajes breves. Tu única tarea es extraer información explícita del mensaje.\n"
                            "Responde **solo** con el valor solicitado. No añadas introducciones, explicaciones, encabezados, ni texto adicional.\n"
                            "No uses formato, como negritas, cursivas o cualquier otro tipo de markdown.\n"
                            "Utiliza el formato específico.\n"
                            "Esfuerzate al 100%.\n"
                            "Los resultados son muy importantes para mi.\n"
    
                         f"""Determina si en el texto se están comparando productos entre productos de Mercadona o de la marca Hacendado y otras marcas y posibles competidores.

                            -Instrucciones:
                            1. Detectar comparación: Lee el texto y comprueba si se mencionan al menos dos productos en 'Producto'.
                            2. Determinar si existe una comparativa entre productos:
                               - para ello usa coomo contexto  el 'Texto' y la 'Justificación 1' y la 'Justificación 2' para determinar el sentimiento hacia un producto de mercadona o hacendado que se este comparando con otro.        
                            3. Regla final:
                               - Si se detecta comparación y uno de los productos es de Mercadona/Hacendado responde con 'SI' o 'NO'.
                            4. Formato de respuesta:
                               - Devuelve solo una de estas opciones: SI o NO.
                               - No añadas explicaciones ni texto adicional.
                            
                            -Ejemplos:
                            
                            Ejemplo 1:
                            Producto: '1: Bephantol 2: vaselina'
                            Texto: 'Bephantol 8,95 en farmacia, vaselina 2 en Mercadona, yo no me la juego'
                            Justificación 1: 'Sentimiento: negativo. Razón: El mensaje expresa una preferencia por la vaselina por ser más económica que el Bephantol, lo que implica una desaprobación del precio del Bephantol.'
                            Justificación 2: 'Sentimiento: negativo. Razón: El mensaje expresa una preferencia clara por la vaselina por su bajo precio, implicando que el Bephantol es caro y no vale la pena.'
                            Respuesta: SI
                 
                            -Ahora analiza el siguiente caso:
                            
                            Producto: '{producto}'
                            Texto: '{documento}'
                            Justificación 1: '{explica_sentimiento}'
                            Justificación 2: '{explica_sentimiento_2}'
                            Respuesta:"""
        
                        
                 )}]
        
        
                    input_ids = tokenizer.apply_chat_template(prompt_comparativa_producto, return_tensors="pt", return_dict=True, add_generation_prompt=True)    
                    input_ids = {k: v.to('cuda:0') for k, v in input_ids.items()}
                    outputs = model.generate(**input_ids, max_new_tokens=50, do_sample=False) 
                    response_comparativa_producto = tokenizer.decode(outputs[0],skip_special_tokens=True)
        
                    #print(response_comparativa_producto)
        
                    comparativa_producto .append(response_comparativa_producto)       


            elif i == 10 :
                if  comparativa_producto[-1].strip()  == 'NO' or comparativa_producto[-1].strip() == 'No aplica':
                    comparativa_sentimiento_producto.append('No aplica') 
                else:
                    prompt_comparativa_sentimiento_producto = [{
                        "role": "user",
                        "content": (
                            "Eres un experto en mensajes breves. Tu única tarea es extraer la información solicitada del mensaje.\n"
                            "Responde **solo** con el valor solicitado. No añadas introducciones, explicaciones, encabezados, ni texto adicional.\n"
                            "No uses formato, como negritas, cursivas o cualquier otro tipo de markdown.\n"
                            "Utiliza el formato específico.\n"
                            "Esfuerzate al 100%.\n"
                            "Los resultados son muy importantes para mi.\n"
                            f""" valora el producto de mercadona o hacendado respecto a producto que se compara. 
                               - Instrucciones:
                                    1. Determina cuál es el producto de la marca mercadona o hacendado.
                                    2. valora el producto de mercadona o hacendado respecto a su competidor. 
                                        - Respuestas posibles: positivo, negativo o neutral.
                                    3. Formato de respuesta:
                                        - Producto de mercadona o hacendado : valoracion del producto respecto a la competencia. ej: yogur de stracciatella : positivo
                            
                            Texto: '{documento}'
                            Respuesta:"""

                            )}]

            
                    
                    input_ids = tokenizer.apply_chat_template(prompt_comparativa_sentimiento_producto, return_tensors="pt", return_dict=True, add_generation_prompt=True)    
                    input_ids = {k: v.to('cuda:0') for k, v in input_ids.items()}
                    outputs = model.generate(**input_ids, max_new_tokens=50, do_sample=False) 
                    response_comparativa_sentimiento_producto = tokenizer.decode(outputs[0],skip_special_tokens=True)

                    #print("COMPARATIVA")
                    #print(limpiar_numeros(producto[-1]))
                    #print(response_comparativa_sentimiento_producto)
        
                    comparativa_sentimiento_producto.append(response_comparativa_sentimiento_producto)       


    
    # resultados linea a dataframe
    temp_df['explica_sentimiento'] = explica_sentimiento
    temp_df['explica_sentimiento_2'] = explica_sentimiento_2
    temp_df['sentimiento_final'] = sentimiento_final  ###
    temp_df['proteccion_datos'] = LOPD
    temp_df['deteccion_producto'] = deteccion_producto ###
    temp_df['producto'] = producto
    temp_df['sentimiento_producto'] = sentimiento_producto ####
    temp_df['categoria'] = categoria
    temp_df['comparativa_producto'] = comparativa_producto 
    temp_df['comparativa_sentimiento'] = comparativa_sentimiento_producto
    temp_df['imagen_marca'] = imagen_marca ###


    # Añadir al DataFrame general
    df_resultado = pd.concat([df_resultado, temp_df], ignore_index=True)



In [ ]:
#función de limpieza de resultados del LLM
def clean_cell(text):
    return text.split('\n')[0]

columns = ['proteccion_datos', 'deteccion_producto','sentimiento_final', 'sentimiento_producto','imagen_marca', 'comparativa_producto', 'comparativa_sentimiento']  #'producto',


#aplicar
for col in df_resultado[columns]:
    df_resultado[col] = df_resultado[col].apply(clean_cell)



#df_resultado['producto'] = df_resultado['producto'].str.split("\n")
df_resultado['producto'] = df_resultado['producto'].str.replace("\n", " ")
df_resultado


In [ ]:
df_resultado['deteccion_producto'].value_counts()
    

In [ ]:
df_final_LLM = pd.concat([df.reset_index(), df_resultado], axis=1)
#df_final_LLM.to_excel('.xlsx')
df_final_LLM

In [ ]:
df_final_LLM.info()

In [ ]:
df_final_LLM.describe(exclude='int64')

In [ ]:
lista=[ 'sentimiento_final', 'search', 'proteccion_datos', 'deteccion_producto', 'sentimiento_producto', 'comparativa_producto', 'imagen_marca' ]

for i in df_final_LLM[lista]:
  print('=================')
  print(df_final_LLM[i].value_counts())
  print('=================')

In [ ]:
df_limp = df_final_LLM['tweets'][df_final_LLM['proteccion_datos'].str.upper().astype(str).str.strip() == 'SI']

for i in df_limp:
    print(i)

In [ ]:
### ver los resultados de sentimiento_final  y arreglarlos

df_limp = df_final_LLM['tweets'][df_final_LLM['sentimiento_final'] == 'sarcástico']

for i in df_limp:
    print(i)
    

df_limp = df_final_LLM.loc[df_final_LLM['sentimiento_final'] == 'sarcástico', ['tweets']]

resultado = resultado = pd.merge(
    df_limp,
    df_final_LLM,
    left_index=True,
    right_index=True,
    how='left'
)

resultado

In [ ]:
df_limp = df_final_LLM['tweets'][df_final_LLM['sentimiento_final'] == 'irónico']

for i in df_limp:
    print(i)

In [ ]:
import re
def limpiar_sarcastico(df):
    def extraer_sentimiento(row):
        if row['sentimiento_final'] == 'sarcástico':
            # Buscar patrón 'Sentimiento: <valor>.'
            match = re.search(r"Sentimiento:\s*(\w+)\.", str(row['explica_sentimiento_2']))
            if match:
                return match.group(1).lower()  # Devuelve el sentimiento en minúscula
            else:
                return row['sentimiento_final'] 
        else:
            return row['sentimiento_final']
    
    df['sentimiento_final'] = df.apply(extraer_sentimiento, axis=1)
    return df
    

In [ ]:
df_final_LLM['sentimiento_final'] = df_final_LLM['sentimiento_final'].str.lower().astype(str).str.strip()
df_final_LLM['sentimiento_final']  = df_final_LLM['sentimiento_final'] .str.replace('negative', 'negativo')
df_final_LLM['sentimiento_final']  = df_final_LLM['sentimiento_final'] .str.replace('posiivo', 'positivo')
df_final_LLM['sentimiento_final']  = df_final_LLM['sentimiento_final'] .str.replace('positivio', 'positivo')
df_final_LLM['sentimiento_final']  = df_final_LLM['sentimiento_final'] .str.replace('irónico', 'negativo')

df_final_LLM = limpiar_sarcastico(df_final_LLM)

df_final_LLM['sentimiento_producto'] = df_final_LLM['sentimiento_producto'].str.upper().astype(str).str.strip()
df_final_LLM['deteccion_producto'] = df_final_LLM['deteccion_producto'].str.replace('.', '', regex=False)

#df.loc[[888,1072 ],'sentimiento_final'] =  ['positivo' , 'neutral']

In [ ]:
lista=[ 'sentimiento_final', 'search', 'proteccion_datos', 'deteccion_producto', 'sentimiento_producto', 'comparativa_producto', 'imagen_marca' ]

for i in df_final_LLM[lista]:
  print('=================')
  print(df_final_LLM[i].value_counts())
  print('=================')

In [ ]:
len(df_final_LLM)

In [ ]:
df_final_LLM = df_final_LLM[df_final_LLM['proteccion_datos'].str.upper().str.strip() != 'SI']
len(df_final_LLM)

In [ ]:
df_final_LLM.to_parquet( './data/Silver/data_for_trainning_encoder.parquet' ,index=False)